In [ ]:
# Workaround to install pystruct.
!pip install --upgrade pip setuptools==57.5.0 
!git clone https://github.com/pystruct/pystruct && cd pystruct
!curl https://patch-diff.githubusercontent.com/raw/pystruct/pystruct/pull/221.patch | git apply
!pip install pystruct/

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 6.4 MB/s 
     |████████████████████████████████| 819 kB 62.5 MB/s 
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.9.0 requires jedi>=0.10, which is not installed.


Cloning into 'pystruct'...
remote: Enumerating objects: 6974, done.
remote: Total 6974 (delta 0), reused 0 (delta 0), pack-reused 6974
Receiving objects: 100% (6974/6974), 12.16 MiB | 15.07 MiB/s, done.
Resolving deltas: 100% (4357/4357), done.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1260k    0 1260k    0     0  1855k      0 --:--:-- --:--:-- --:--:-- 1853k
<stdin>:912: trailing whitespace.
 * 
<stdin>:915: trailing whitespace.
 * 
<stdin>:2642: trailing whitespace.
      __pyx_t_3 = __Pyx_TypeCheck(__pyx_v_arg, __pyx_v_ndarray); 
<stdin>:2652: trailing whitespace.
      __pyx_t_2 = __pyx_memoryview_check(__pyx_v_arg); 
<stdin>:2660: trailing whitespace.
        __pyx_t_3 = __Pyx_TypeCheck(__pyx_v_arg_base, __pyx_v_ndarray); 
error: src/utils.c: No such file or directory
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simpl

In [ ]:
from tqdm.notebook import tqdm
import json
import cv2
import numpy as np
import os
from pystruct.models import EdgeFeatureGraphCRF
from pystruct.learners import FrankWolfeSSVM
from pystruct.utils import SaveLogger
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D, Dense, MaxPooling2D, Flatten
from tensorflow.keras.optimizers import SGD
from keras.metrics import sparse_categorical_crossentropy, categorical_crossentropy
from google.colab import drive

drive.mount("/content/drive")


train_image_folder_path = "drive/MyDrive/btp-dataset/Train/Train/Image/"
train_annotation_folder_path = "drive/MyDrive/btp-dataset/Train/Train/Annotations/"
train_prepared_annotation_folder_path = "drive/MyDrive/btp-dataset/Train/Prepared/Annotations/"
testing_image_folder_path = "drive/MyDrive/btp-dataset/Train/Train/Image/"
testing_annotation_folder_path = "drive/MyDrive/btp-dataset/Train/Train/Annotations/"
saved_unary_model_path = "drive/MyDrive/btp-dataset/SavedModels/unary/"
saved_pairwise_model_path = "drive/MyDrive/btp-dataset/SavedModels/pairwise/"
saved_crf_model_path = "drive/MyDrive/btp-dataset/SavedModels/crf/"


def scale_img_for_unary(img, x1, x2, y1, y2):

    width = x2 - x1 + 1
    height = y2 - y1 + 1

    final_img = img[y1 : y2 + 1, x1 : x2 + 1]

    if width <= 640 and height <= 32:
        vertical_padding = 32 - height
        horizontal_padding = 640 - width
        final_img = cv2.copyMakeBorder(
            final_img,
            vertical_padding // 2,
            vertical_padding // 2 + vertical_padding % 2,
            horizontal_padding // 2,
            horizontal_padding // 2 + horizontal_padding % 2,
            cv2.BORDER_CONSTANT,
            None,
            (255, 255, 255),
        )

    elif height <= 32:
        scale_down = width / 640
        adjusted_height = int(height / scale_down)
        final_img = cv2.resize(final_img, (640, adjusted_height))

        vertical_padding = 32 - adjusted_height

        final_img = cv2.copyMakeBorder(
            final_img,
            vertical_padding // 2,
            vertical_padding // 2 + vertical_padding % 2,
            0,
            0,
            cv2.BORDER_CONSTANT,
            None,
            (255, 255, 255),
        )

    elif width <= 640:
        scale_down = height / 32
        adjusted_width = int(width / scale_down)
        final_img = cv2.resize(final_img, (adjusted_width, 32))

        horizontal_padding = 640 - adjusted_width

        final_img = cv2.copyMakeBorder(
            final_img,
            0,
            0,
            horizontal_padding // 2,
            horizontal_padding // 2 + horizontal_padding % 2,
            cv2.BORDER_CONSTANT,
            None,
            (255, 255, 255),
        )

    else:
        scale_down_by_width = width / 640
        scale_down_by_height = height / 32

        if scale_down_by_height < scale_down_by_width:
            adjusted_height = int(height / scale_down_by_width)
            final_img = cv2.resize(final_img, (640, adjusted_height))
            vertical_padding = 32 - adjusted_height

            final_img = cv2.copyMakeBorder(
                final_img,
                vertical_padding // 2,
                vertical_padding // 2 + vertical_padding % 2,
                0,
                0,
                cv2.BORDER_CONSTANT,
                None,
                (255, 255, 255),
            )

        else:
            adjusted_width = int(width / scale_down_by_height)
            final_img = cv2.resize(final_img, (adjusted_width, 32))
            horizontal_padding = 640 - adjusted_width

            final_img = cv2.copyMakeBorder(
                final_img,
                0,
                0,
                horizontal_padding // 2,
                horizontal_padding // 2 + horizontal_padding % 2,
                cv2.BORDER_CONSTANT,
                None,
                (255, 255, 255),
            )

    return final_img


def scale_img_for_pairwise(img, x1, x2, y1, y2):

    width = x2 - x1 + 1
    height = y2 - y1 + 1

    final_img = img[y1 : y2 + 1, x1 : x2 + 1]

    if width <= 640 and height <= 16:
        vertical_padding = 16 - height
        horizontal_padding = 640 - width
        final_img = cv2.copyMakeBorder(
            final_img,
            vertical_padding // 2,
            vertical_padding // 2 + vertical_padding % 2,
            horizontal_padding // 2,
            horizontal_padding // 2 + horizontal_padding % 2,
            cv2.BORDER_CONSTANT,
            None,
            (255, 255, 255),
        )

    elif height <= 16:
        scale_down = width / 640
        adjusted_height = int(height / scale_down)
        final_img = cv2.resize(final_img, (640, adjusted_height))

        vertical_padding = 16 - adjusted_height

        final_img = cv2.copyMakeBorder(
            final_img,
            vertical_padding // 2,
            vertical_padding // 2 + vertical_padding % 2,
            0,
            0,
            cv2.BORDER_CONSTANT,
            None,
            (255, 255, 255),
        )

    elif width <= 640:
        scale_down = height / 16
        adjusted_width = int(width / scale_down)
        final_img = cv2.resize(final_img, (adjusted_width, 16))

        horizontal_padding = 640 - adjusted_width

        final_img = cv2.copyMakeBorder(
            final_img,
            0,
            0,
            horizontal_padding // 2,
            horizontal_padding // 2 + horizontal_padding % 2,
            cv2.BORDER_CONSTANT,
            None,
            (255, 255, 255),
        )

    else:
        scale_down_by_width = width / 640
        scale_down_by_height = height / 16

        if scale_down_by_height < scale_down_by_width:
            adjusted_height = int(height / scale_down_by_width)
            final_img = cv2.resize(final_img, (640, adjusted_height))
            vertical_padding = 16 - adjusted_height

            final_img = cv2.copyMakeBorder(
                final_img,
                vertical_padding // 2,
                vertical_padding // 2 + vertical_padding % 2,
                0,
                0,
                cv2.BORDER_CONSTANT,
                None,
                (255, 255, 255),
            )

        else:
            adjusted_width = int(width / scale_down_by_height)
            final_img = cv2.resize(final_img, (adjusted_width, 16))
            horizontal_padding = 640 - adjusted_width

            final_img = cv2.copyMakeBorder(
                final_img,
                0,
                0,
                horizontal_padding // 2,
                horizontal_padding // 2 + horizontal_padding % 2,
                cv2.BORDER_CONSTANT,
                None,
                (255, 255, 255),
            )

    return final_img


def get_unary_class(u):
    if u == "figureRegion":
        return 0
    elif u == "tableRegion":
        return 1
    elif u == "formulaRegion":
        return 2
    else:
        return 3


def get_unary_data_for_page(annotation_filename, cnn=False):
    train_samples = []
    train_labels = []

    image_filename = annotation_filename[:-4] + "bmp"
    img = cv2.imread(train_image_folder_path + image_filename)

    with open(train_prepared_annotation_folder_path + annotation_filename, "r") as f:
        data = json.load(f)

        for line in data["lines"]:
            try:
                x1, y1, x2, y2 = line["x1"], line["y1"], line["x2"], line["y2"]
                scaled_line_img = scale_img_for_unary(img, x1, x2, y1, y2)
                train_samples.append(scaled_line_img)
                train_labels.append(get_unary_class(line["block_class"]))

                if cnn and get_unary_class(line["block_class"]) != get_unary_class(
                    "textRegion"
                ):
                    train_samples.append(cv2.flip(scaled_line_img, 0))
                    train_labels.append(get_unary_class(line["block_class"]))
                    train_samples.append(cv2.flip(scaled_line_img, 1))
                    train_labels.append(get_unary_class(line["block_class"]))
                    train_samples.append(cv2.flip(scaled_line_img, -1))
                    train_labels.append(get_unary_class(line["block_class"]))
            except Exception as e:
                print(e)
                pass

    return train_samples, train_labels


def get_pairwise_class(u, v):
    if v == "textRegion" and u == "tableRegion":
        return 0
    elif v == "textRegion" and u == "formulaRegion":
        return 1
    elif v == "textRegion" and u == "textRegion":
        return 2
    elif v == "textRegion" and u == "figureRegion":
        return 3
    elif v == "tableRegion" and u == "tableRegion":
        return 4
    elif v == "tableRegion" and u == "formulaRegion":
        return 5
    elif v == "tableRegion" and u == "textRegion":
        return 6
    elif v == "tableRegion" and u == "figureRegion":
        return 7
    elif v == "figureRegion" and u == "tableRegion":
        return 8
    elif v == "figureRegion" and u == "formulaRegion":
        return 9
    elif v == "figureRegion" and u == "textRegion":
        return 10
    elif v == "figureRegion" and u == "figureRegion":
        return 11
    elif v == "formulaRegion" and u == "tableRegion":
        return 12
    elif v == "formulaRegion" and u == "formulaRegion":
        return 13
    elif v == "formulaRegion" and u == "textRegion":
        return 14
    elif v == "formulaRegion" and u == "figureRegion":
        return 15
    else:
        return -1


def get_pairwise_data_for_page(annotation_filename):
    train_samples = []
    train_labels = []
    image_filename = annotation_filename[:-4] + "bmp"
    img = cv2.imread(train_image_folder_path + image_filename)

    with open(train_prepared_annotation_folder_path + annotation_filename, "r") as f:
        data = json.load(f)
        line_list = data["lines"]

        for i in range(len(line_list) - 1):
            try:
                y11, x11 = line_list[i]["y1"], line_list[i]["x1"]
                y12, x12 = line_list[i]["y2"], line_list[i]["x2"]
                y21, x21 = line_list[i + 1]["y1"], line_list[i + 1]["x1"]
                y22, x22 = line_list[i + 1]["y2"], line_list[i + 1]["x2"]
                scaled_img1 = scale_img_for_pairwise(img, x11, x12, y11, y12)
                scaled_img = scale_img_for_pairwise(img, x21, x22, y21, y22)
                scaled_img1 = cv2.resize(img[y11 : y12 + 1, x11 : x12 + 1], (640, 16))
                scaled_img2 = cv2.resize(img[y21 : y22 + 1, x21 : x22 + 1], (640, 16))
                scaled_line_img = cv2.vconcat([scaled_img1, scaled_img2])

                train_samples.append(scaled_line_img)
                train_labels.append(
                    get_pairwise_class(
                        line_list[i]["block_class"], line_list[i + 1]["block_class"]
                    )
                )
            except Exception as e:
                pass
    return train_samples, train_labels


def get_unary_potentials(x):
    unary_potentials = []
    unary = tf.keras.models.load_model(saved_unary_model_path)
    for t in x:
        t = np.expand_dims(t, axis=0)
        p = unary.predict(t, verbose=0)
        unary_potentials.append(np.array(p[0]))
        # print(np.shape(p))
    return unary_potentials


def get_pairwise_potentials(x):
    pairwise_potentials = []
    pairwise = tf.keras.models.load_model(saved_pairwise_model_path)
    for t in x:
        t = np.expand_dims(t, axis=0)
        pairwise_potentials.append(np.array(pairwise.predict(t, verbose=0)[0]))
    return pairwise_potentials


def get_crf_training_data():
    # We have 1600 files, first 1280 are for training.
    x_train = []
    y_train = []

    list_of_files = os.listdir(train_prepared_annotation_folder_path)
    total_number_of_files = 300
    file_counter = 0
    with tqdm(total=total_number_of_files) as pbar:
        for i in range(total_number_of_files):
            annotation_filename = list_of_files[i]
            file_counter += 1

            x_train_u, y_train_u = get_unary_data_for_page(
                annotation_filename, cnn=False
            )
            x_train_p, _ = get_pairwise_data_for_page(annotation_filename)
            unary_potential_list = np.array(get_unary_potentials(x_train_u))

            pairwise_potential_list = np.array(get_pairwise_potentials(x_train_p))


            n_nodes = len(unary_potential_list)
            edges = np.transpose(
                np.vstack([np.arange(n_nodes - 1), np.arange(1, n_nodes)])
            )

            if (
                np.shape(unary_potential_list)[0] - 1
                == np.shape(pairwise_potential_list)[0]
                and np.shape(y_train_u)[0] == np.shape(unary_potential_list)[0]
            ):

                x_train.append((unary_potential_list, edges, pairwise_potential_list))
                y_train.append(np.array(y_train_u))

            pbar.update(1)

    return x_train, y_train

Mounted at /content/drive


In [ ]:
crf = EdgeFeatureGraphCRF(n_features=4, n_edge_features=16, inference_method="max-product")
ssvm = FrankWolfeSSVM(model=crf, C=0.1, max_iter=11)
x_train, y_train = get_crf_training_data()

  0%|          | 0/300 [00:00<?, ?it/s]

OpenCV(4.6.0) /io/opencv/modules/imgproc/src/resize.cpp:4055: error: (-215:Assertion failed) inv_scale_x > 0 in function 'resize'

OpenCV(4.6.0) /io/opencv/modules/imgproc/src/resize.cpp:4055: error: (-215:Assertion failed) inv_scale_x > 0 in function 'resize'

OpenCV(4.6.0) /io/opencv/modules/imgproc/src/resize.cpp:4055: error: (-215:Assertion failed) inv_scale_x > 0 in function 'resize'

OpenCV(4.6.0) /io/opencv/modules/imgproc/src/resize.cpp:4055: error: (-215:Assertion failed) inv_scale_x > 0 in function 'resize'

OpenCV(4.6.0) /io/opencv/modules/imgproc/src/resize.cpp:4055: error: (-215:Assertion failed) inv_scale_x > 0 in function 'resize'

OpenCV(4.6.0) /io/opencv/modules/imgproc/src/resize.cpp:4055: error: (-215:Assertion failed) inv_scale_x > 0 in function 'resize'

OpenCV(4.6.0) /io/opencv/modules/imgproc/src/resize.cpp:4055: error: (-215:Assertion failed) inv_scale_x > 0 in function 'resize'

OpenCV(4.6.0) /io/opencv/modules/imgproc/src/resize.cpp:4055: error: (-215:Assertio

In [ ]:
ssvm.fit(x_train, y_train)

FrankWolfeSSVM(C=0.1, max_iter=11,
               model=EdgeFeatureGraphCRF(n_states: 4, inference_method: max-product, n_features: 4, n_edge_features: 16))

In [ ]:
y_train_flat = [y.ravel() for y in y_train]

In [ ]:
# from pystruct.models import ChainCRF
# from pystruct.learners import FrankWolfeSSVM
# model = ChainCRF()
# ssvm = FrankWolfeSSVM(model=model, C=.1, max_iter=10)

In [ ]:
# print(np.shape(x_train))
# print(np.shape(y_train))
# print(np.shape(x_train[0]))
# print(np.shape(y_train[0]))
# print(x_train[0])

# from pystruct.datasets import load_letters
# letters = load_letters()
# X, y, folds = letters['data'], letters['labels'], letters['folds']
# X, y = np.array(X), np.array(y)
# X_train, X_test = X[folds == 1], X[folds != 1]
# y_train, y_test = y[folds == 1], y[folds != 1]
# from pystruct.models import ChainCRF
# from pystruct.learners import FrankWolfeSSVM
# model = ChainCRF()
# ssvm = FrankWolfeSSVM(model=model, C=.1, max_iter=10)
# ssvm.fit(X_train, y_train) 

In [ ]:
np.shape(y_train[10])

(7,)

In [ ]:
sv = SaveLogger('./crf/')
sv.save(ssvm, saved_crf_model_path+'crf_model')

In [ ]:
def get_crf_testing_data():
  # We have 1600 files, first 1280 are for training.
  x_test = []
  y_test = []

  list_of_files = os.listdir(train_prepared_annotation_folder_path)
  total_number_of_files = 10
  file_counter = 0
  with tqdm(total=total_number_of_files) as pbar:
    for i in range(total_number_of_files):
      annotation_filename = list_of_files[1280+i]
      file_counter += 1

      x_test_u, y_test_u = get_unary_data_for_page(annotation_filename, cnn=False)
      x_test_p, _ = get_pairwise_data_for_page(annotation_filename)
      unary_potential_list = np.array(get_unary_potentials(x_test_u))
      pairwise_potential_list = np.array(get_pairwise_potentials(x_test_p))
      
      n_nodes = len(unary_potential_list)
      edges = np.transpose(np.vstack([np.arange(n_nodes - 1), np.arange(1, n_nodes)]))
      if(np.shape(unary_potential_list)[0] - 1 == np.shape(pairwise_potential_list)[0] and np.shape(y_test_u)[0]==np.shape(unary_potential_list)[0]):

        x_test.append((unary_potential_list, edges, pairwise_potential_list))
        y_test.append(np.array(y_test_u))
    
      pbar.update(1)

  return x_test, y_test

In [ ]:
sv = SaveLogger(saved_crf_model_path+'crf_model')
ssvm = sv.load()
x_test, y_test = get_crf_testing_data()
score = ssvm.score(x_test, y_test)
print(score)
y = ssvm.predict(x_test)

  0%|          | 0/10 [00:00<?, ?it/s]

OpenCV(4.6.0) /io/opencv/modules/imgproc/src/resize.cpp:4055: error: (-215:Assertion failed) inv_scale_x > 0 in function 'resize'

0.9090909090909091


In [ ]:
print(y)
print(np.shape(y[2]))
print(np.shape(x_test[2][0]))

[array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]), array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]), array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]), array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]), array([3, 3, 3, 3, 3, 3, 3, 3, 3]), array([3, 3, 3, 3]), array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]), array([3, 3, 3, 3]), array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3])]
(42,)
(42, 4)


In [ ]:
def classify_page_objects(human_read_lines, img):
  x_unary = get_unary_lines(human_read_lines, img)
  x_pairwise = get_pairwise_lines(human_read_lines, img)
  unary = tf.keras.models.load_model(saved_unary_model_path)
  pairwise = tf.keras.models.load_model(saved_pairwise_model_path)
  sv = SaveLogger(saved_crf_model_path)
  ssvm = sv.load()
  unary_potential_list = np.array(unary.predict(x_unary))
  pairwise_potential_list = np.array(pairwise.predict(x_pairwise))
  
  n_nodes = len(unary_potential_list)
  edges = np.transpose(np.vstack([np.arange(n_nodes - 1), np.arange(1, n_nodes)]))

  x_data = np.array([unary_potential_list, edges, pairwise_potential_list])

  y_data = ssvm.predict(x_data)
  return human_read_lines